In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama


In [5]:
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)


In [2]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]


In [3]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples=examples,   # Available examples
    embeddings=OllamaEmbeddings(model="nomic-embed-text:latest"),  # Embedding model
    vectorstore_cls=FAISS,  # FAISS for fast retrieval
    k=2,  # Retrieve the 2 most relevant yet diverse examples
)


In [ ]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)


In [8]:
llm = ChatOllama(model="llama3.2:latest")


In [ ]:
print(mmr_prompt.format(adjective="worried"))


In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OllamaEmbeddings(model="nomic-embed-text:latest"),
    vectorstore_cls=FAISS,
    k=2,  # Retrieve top 2 similar examples
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

print(similar_prompt.format(adjective="worried"))
